In [43]:
import torch

import pyprob
from pyprob import Model
import pyprob.distributions as dists

from calendar import monthrange
import datetime

In [55]:
class DateParser(Model):
    def __init__(self):
        super().__init__(name="Date with Unkown Format")
        
    def forward(self):
        # sample if year is in near future, near past, or distant past/future
        year_type = pyprob.sample(dists.Categorical(torch.tensor([0.5, 0.4, 0.1]))).item()
        year_type = ["near future", "near_past", "distant"]
        if year_type is "near future":
            year = 2019 + int(pyprob.sample(dists.Categorical(torch.tensor([0.01]*100))))
        elif year_type is "near future":
            year = 2019 - int(pyprob.sample(dists.Categorical(torch.tensor([0.01]*100))))
        else:
            year_1 = int(pyprob.sample(dists.Categorical(torch.tensor([0.1]*10))).item())
            year_2 = int(pyprob.sample(dists.Categorical(torch.tensor([0.1]*10))).item())
            year_3 = int(pyprob.sample(dists.Categorical(torch.tensor([0.1]*10))).item())
            year_4 = int(pyprob.sample(dists.Categorical(torch.tensor([0.1]*10))).item())
            year = int("".join(str(d) for d in [year_1, year_2, year_3, year_4]))

        month = int(pyprob.sample(dists.Categorical(torch.tensor([1/12]*12))).item())
        num_days = monthrange(year, month)                         # number of days in this month
        day_probs = [1/num_days]*num_days + [0.]*(31-num_days)     # probs of which day it is (in fixed length vector)
        day = pyprob.sample(dists.Categorical(torch.tensor(day_probs)))

        yy = pyprob.sample(dists.Categorical(torch.tensor([0.5, 0.5]))).item()  # either yy or yyyy
        m = pyprob.sample(dists.Categorical(torch.tensor([0.5, 0.5]))).item()   # either m or mm
        d = pyprob.sample(dists.Categorical(torch.tensor([0.5, 0.5]))).item()   # either d or dd
        
        return f"{day}/{month}/{year}"
        


In [56]:
a = dists.Categorical(torch.tensor([1/12]*12))

In [58]:
DateParser().prior_traces(1)

Time spent  | Time remain.| Progress             | Trace | Traces/sec


TypeError: unsupported operand type(s) for /: 'int' and 'tuple'